# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [32]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [7]:
weather_df = pd.read_csv("../resources/city_weather.csv")
weather_df.head()

,Unnamed: 0,City,Lat,Lon,Temperature,Min Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,banda aceh,5.552846,95.319291,85.89,85.89,85.89,67,97,10.76,ID,1651459094
1,1,pacific grove,36.621115,-121.917794,54.21,51.67,63.05,81,0,10.36,US,1651458821
2,2,constitucion,-35.331831,-72.411900,52.29,52.29,52.29,73,2,6.62,CL,1651459095
3,3,hermanus,-34.417500,19.236111,59.85,59.85,59.85,80,99,9.15,ZA,1651459096
4,4,thompson,55.743347,-97.863464,24.98,24.98,24.98,86,100,13.80,CA,1651459096


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [77]:
locations = weather_df[["Lat","Lon"]]
humidity = weather_df["Humidity"]

gmaps.configure(api_key=g_key)
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=250,
                                 point_radius=5)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
ideal_min_temp = 60
ideal_max_temp = 70
ideal_max_cloud = 20
ideal_wind_speed = 10
ideal_humidity = 50
ideal_weather_df = weather_df.loc[
    (weather_df["Min Temp"] >= ideal_min_temp) & 
    (weather_df["Max Temp"] <= ideal_max_temp) &
    (weather_df["Cloudiness"] <= ideal_max_cloud) &
    (weather_df["Wind Speed"] <= ideal_wind_speed) &
    (weather_df["Humidity"] <= ideal_humidity)
]
ideal_weather_df

,Unnamed: 0,City,Lat,Lon,Temperature,Min Temp,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
185,185,pingliang,35.543200,106.660811,61.36,61.36,61.36,16,0,8.97,CN,1651459212
234,234,sakakah,29.978401,40.204770,67.57,67.57,67.57,39,0,7.11,SA,1651459244
364,364,sayyan,15.171828,44.325428,60.48,60.48,60.48,44,15,3.98,YE,1651459328
367,367,prince george,53.912864,-122.745370,60.21,60.21,60.21,33,0,9.22,CA,1651459330
413,413,port pirie,-33.179125,138.005861,69.85,69.85,69.85,46,0,4.99,AU,1651459359
479,479,manali,32.245461,77.187293,65.48,65.48,65.48,46,0,3.09,IN,1651459405
531,531,jiuquan,39.732126,98.493684,62.87,62.87,62.87,13,7,6.11,CN,1651459436
541,541,korla,41.723874,86.173541,67.14,67.14,67.14,19,9,9.80,CN,1651459443
571,571,zhangye,38.936513,100.455633,61.30,61.30,61.30,15,0,7.36,CN,1651459462


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [63]:
hotel_dict = {
    "Hotel Name": [],
    "Lat": [],
    "Lng": [],
    "City": [],
    "Country": []
}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in ideal_weather_df.iterrows():
    lat = row["Lat"]
    lng = row["Lon"]
    constructed_url = base_url + f"?key={g_key}&location={lat},{lng}&radius=5000&type=lodging"
    response = requests.get(constructed_url).json()
    hotel_dict["Hotel Name"].append(response["results"][0]["name"])
    hotel_dict["Lat"].append(response["results"][0]["geometry"]["location"]["lat"])
    hotel_dict["Lng"].append(response["results"][0]["geometry"]["location"]["lng"])
    hotel_dict["City"].append(row["City"])
    hotel_dict["Country"].append(row["Country"])

hotel_df = pd.DataFrame(hotel_dict)    

In [64]:
hotel_df

,Hotel Name,Lat,Lng,City,Country
0,Mintai Hostel,35.548703,106.662289,pingliang,CN
1,Fakhamat Al Orjoana Apartment,29.946421,40.193125,sakakah,SA
2,متنزه و بحيرة سيان,15.186530,44.327739,sayyan,YE
3,Canadas Best Value Inn Prince George,53.909205,-122.780473,prince george,CA
4,John Pirie Motor Inn,-33.186513,138.025739,port pirie,AU
5,Tourist Hotel,32.248186,77.185501,manali,IN
6,Jiuquan Hotel,39.737397,98.507802,jiuquan,CN
7,Gulisitan Hotel,41.753048,86.144840,korla,CN
8,Zhangye Lvzhou Holiday Inn,38.933208,100.428128,zhangye,CN


'Mintai Hostel'
35.54870349999999
106.6622885


In [78]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [79]:
# Add marker layer ontop of heat map
fig.add_layer(gmaps.symbol_layer(locations, info_box_content=hotel_info))
fig
# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…